                                                                            Luis Ramirez Camargo, June 2020

# Calculate accuracy indicators for PV output calculated with ERA5-land and MERRA 2 Data  

This notebook uses the outputs from the notebook clean_measured_pv_data_installations_chile to evaluate the accuracy of the outputs from  the notebook pv_output_from_ERA5_land_and_merra2. The accuracy is assessed using the pearsons coorelation coeficient, the Mean Biased Error (MBE), the root mean square error (rmse). 


## 1) import the libraries and data

In [1]:
import os
import itertools
import xarray as xr
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import glob
from tkinter import Tcl
import gc
import scipy as sp
import scipy.stats
import seaborn as sns
from pathlib import Path

## Import the capacity factors of the meassured data

In [2]:
reference = pd.read_csv(Path('intermediate_results/time_series_PV_sen_chile_capacity_factors.csv'),
                        index_col=0, parse_dates=True)
#errase the erroneous part of the time series of installations that despite erroneous shape
#still have more than one year of meassurements and can be taken into account for comparison
reference.loc[:'2017-09','SOLAR HORMIGA'] = np.nan
reference.loc[:'2017-11','SOLAR EL AGUILA I'] = np.nan

In [3]:
reference_optimal_des_rad = pd.read_csv(Path('data/deseason_rad_optimal_reference.csv'),
                        index_col=0, parse_dates=True)
reference_tracking_des_rad = pd.read_csv(Path('data/deseason_rad_tracking_reference.csv'),
                        index_col=0, parse_dates=True)
#errase the erroneous part of the time series of installations that despite erroneous shape
#still have more than one year of meassurements and can be taken into account for comparison
reference_optimal_des_rad.loc[:'2017-09','SOLAR HORMIGA'] = np.nan
reference_optimal_des_rad.loc[:'2017-11','SOLAR EL AGUILA I'] = np.nan

In [4]:
reference_non_tracking_cumulated = pd.read_csv(Path('intermediate_results/cumulated_non_tracking_reference.csv'),
                        index_col=0, parse_dates=True)
reference_tracking_cumulated = pd.read_csv(Path('intermediate_results/cumulated_tracking_reference.csv'),
                        index_col=0, parse_dates=True)

In [5]:
reference_non_tracking_des_cumulated = pd.read_csv(Path('intermediate_results/cumulated_deseason_non_tracking_reference.csv'),
                        index_col=0, parse_dates=True)
reference_tracking_des_cumulated = pd.read_csv(Path('intermediate_results/cumulated_deseason_tracking_reference.csv'),
                        index_col=0, parse_dates=True)

## 3) Import the capacity factors of the PV output calculated with ERA5-land data and MERRA2-data 

In [6]:
#modify the calculated data sets time series to match chilean summer time
#this cannot be performed automatically since the dailight summer time in chile changes from year to year
def summer_time_chile(cf_file):
    cf_utc = pd.read_csv(cf_file, index_col=0, parse_dates=True)
    cf_ut_st_2014 = cf_utc["2014-04-27":"2014-09-07"].shift(periods=-1).copy()
    cf_ut_st_2016 = cf_utc["2016-05-15":"2016-08-14"].shift(periods=-1).copy()
    cf_ut_st_2017 = cf_utc["2017-05-14":"2017-08-14"].shift(periods=-1).copy()
    cf_ut_st_2018 = cf_utc["2018-05-13":"2018-08-12"].shift(periods=-1).copy()
    cf_utc_st = cf_utc.copy()
    cf_utc_st.loc["2014-04-27":"2014-09-07"] = cf_ut_st_2014
    cf_utc_st.loc["2016-05-15":"2016-08-14"] = cf_ut_st_2016 
    cf_utc_st.loc["2017-05-14":"2017-08-14"] = cf_ut_st_2017
    cf_utc_st.loc["2018-05-13":"2018-08-12"] = cf_ut_st_2018
    return cf_utc_st

In [7]:
era5l_optimal = \
summer_time_chile(Path('intermediate_results/timeseries_capacity_factors_pv_optimal_era5l.csv'))
merra2_optimal = \
summer_time_chile(Path('intermediate_results/timeseries_capacity_factors_pv_optimal_merra2.csv'))
era5l_tracking = \
summer_time_chile(Path('intermediate_results/timeseries_capacity_factors_pv_tracking_era5l.csv'))
merra2_tracking = \
summer_time_chile(Path('intermediate_results/timeseries_capacity_factors_pv_tracking_merra2.csv'))

In [8]:
era5l_optimal_cumulated = \
summer_time_chile(Path('intermediate_results/cumulated_non_tracking_era5l.csv'))
merra2_optimal_cumulated = \
summer_time_chile(Path('intermediate_results/cumulated_non_tracking_merra2.csv'))
era5l_tracking_cumulated = \
summer_time_chile(Path('intermediate_results/cumulated_tracking_era5l.csv'))
merra2_tracking_cumulated = \
summer_time_chile(Path('intermediate_results/cumulated_tracking_merra2.csv'))

In [9]:
era5l_optimal_des_rad = \
summer_time_chile(Path('data/deseason_rad_optimal_era5l.csv'))
merra2_optimal_des_rad = \
summer_time_chile(Path('data/deseason_rad_optimal_merra2.csv'))
era5l_tracking_des_rad = \
summer_time_chile(Path('data/deseason_rad_tracking_era5l.csv'))
merra2_tracking_des_rad = \
summer_time_chile(Path('data/deseason_rad_tracking_merra2.csv'))

In [10]:
era5l_optimal_des_rad_cumulated = \
summer_time_chile(Path('intermediate_results/cumulated_deseason_non_tracking_era5l.csv'))
merra2_optimal_des_rad_cumulated = \
summer_time_chile(Path('intermediate_results/cumulated_deseason_non_tracking_merra2.csv'))
era5l_tracking_des_rad_cumulated = \
summer_time_chile(Path('intermediate_results/cumulated_deseason_tracking_era5l.csv'))
merra2_tracking_des_rad_cumulated = \
summer_time_chile(Path('intermediate_results/cumulated_deseason_tracking_merra2.csv'))

## 4) Indicators calculation for the validation ERA5-land

In [11]:
def validate_pv_output_all(reference_values_df,calculated_values_df, name_file_to_store):
    '''calculate the accuracy indicators for each plant where the reference data set is not empty'''
    indicators = pd.DataFrame(columns=("pearson_h",
                                       "mbe_h",
                                       "rmse_h",
                                       "observations",
                                       "pearson_d",
                                       "mbe_d",
                                       "rmse_d",
                                       "days",
                                       "pearson_m",
                                       "mbe_m",
                                       "rmse_m",
                                       "months"), index=calculated_values_df.columns)
    for plant in calculated_values_df.columns:
        if np.sum(reference_values_df[plant]["2014":"2018"]) != 0:
            #print(plant)
            comp_pre = pd.DataFrame(columns=("new","reference"), index=calculated_values_df.index)
            #print(np.array(calculated_values_df.index).size)
            comp_pre["new"] = np.array(calculated_values_df[plant]["2014":"2018"]).copy()
            #print(np.array(reference_values_df[plant]["2014":"2018"].copy()).size)
            #print(reference_values_df[plant]["2014-01-01"])
            comp_pre.loc[3:,"reference"] = reference_values_df[plant][:-5].values
            #print(comp_pre)
            comp = comp_pre.dropna(axis=0).copy()
            #print(comp)
            new = comp["new"]
            reference = comp["reference"]
            pearson = sp.stats.pearsonr(reference.astype(float), new.astype(float))
            indicators.loc[plant]["pearson_h"] = pearson[0]
            indicators.loc[plant]["mbe_h"] = (np.sum(new-reference))/np.sum(reference.size)
            observations = comp.size/comp.columns.size
            indicators.loc[plant]["observations"] = observations
            indicators.loc[plant]["rmse_h"] = np.sqrt((np.sum((new-reference)**2))/(observations))
            #indicators.loc[plant]["rmse_r_h"] = np.sqrt((np.sum((new-reference)**2))/observations)/(np.sum(reference/observations))
            new_avg_day = round(new.resample('D').mean(),4).dropna().copy()
            days = new_avg_day.size
            indicators.loc[plant]["days"] = days
            reference_avg_day = round(reference.astype(float).resample('D').mean(),4).dropna().copy()
            pearson_days = sp.stats.pearsonr(reference_avg_day.astype(float), new_avg_day.astype(float))
            indicators.loc[plant]["pearson_d"] = pearson_days[0]
            indicators.loc[plant]["mbe_d"] = (np.sum(new_avg_day-reference_avg_day))/np.sum(reference_avg_day.size)
            indicators.loc[plant]["rmse_d"] = np.sqrt((np.sum((new_avg_day-reference_avg_day)**2))/(np.sum(reference_avg_day.size)))
            new_avg_month = round(new.resample('M').mean(),4).dropna().copy()
            months = new_avg_month.size 
            if months > 1:
                #print(plant)
                reference_avg_month = round(reference.astype(float).resample('M').mean(),4).dropna().copy()
                pearson_months = sp.stats.pearsonr(reference_avg_month.astype(float), new_avg_month.astype(float))
                indicators.loc[plant]["pearson_m"] = pearson_months[0]
                indicators.loc[plant]["mbe_m"] = (np.sum(new_avg_month-reference_avg_month))/np.sum(reference_avg_month.size)
                indicators.loc[plant]["rmse_m"] = np.sqrt((np.sum((new_avg_month-reference_avg_month)**2))/(np.sum(reference_avg_month.size)))
                indicators.loc[plant]["months"] = months
    #drop all installations where no meassured data is available
    indicators = indicators.dropna().copy()
    #drop all installations where less than 12 months of data are available
    minimum_months = 12
    indicators = indicators.loc[indicators["months"] > minimum_months].drop(["observations",
                                                                              "days",
                                                                              "months"], axis=1).copy()
    indicators.to_csv(name_file_to_store)
    return indicators

In [12]:
#calculate the indicators between raw meassured data and all ERA5 and MERRA2 
#based estimations for tracking and non-tracking systems
indicators_era5l_optimal = validate_pv_output_all(reference,
                                                  era5l_optimal,
                                                  Path('intermediate_results/indicators_era5l_optimal.csv'))
indicators_era5l_tracking = validate_pv_output_all(reference,
                                                  era5l_tracking,
                                                  Path('intermediate_results/indicators_era5l_tracking.csv'))
indicators_merra2_optimal = validate_pv_output_all(reference,
                                                  merra2_optimal,
                                                  Path('intermediate_results/indicators_merra2_optimal.csv'))
indicators_merra2_tracking = validate_pv_output_all(reference,
                                                  merra2_tracking,
                                                  Path('intermediate_results/indicators_merra2_tracking.csv'))

In [13]:
#calculate the indicators between deseasonalized meassured data and all deseasonalized ERA5 and MERRA2 
#based estimations for tracking and non-tracking systems
indicators_era5l_optimal_des_rad = \
validate_pv_output_all(reference_optimal_des_rad,
                       era5l_optimal_des_rad,
                       Path('intermediate_results/indicators_era5l_optimal_deseason_rad.csv'))
indicators_era5l_tracking_des_rad =\
validate_pv_output_all(reference_tracking_des_rad,
                       era5l_tracking_des_rad,
                       Path('intermediate_results/indicators_era5l_tracking_deseason_rad.csv'))
indicators_merra2_optimal_des_rad = \
validate_pv_output_all(reference_optimal_des_rad,
                       merra2_optimal_des_rad,
                       Path('intermediate_results/indicators_merra2_optimal_deseason_rad.csv'))
indicators_merra2_tracking_des_rad = \
validate_pv_output_all(reference_tracking_des_rad,
                       merra2_tracking_des_rad,
                       Path('intermediate_results/indicators_merra2_tracking_deseason_rad.csv'))

In [14]:
indicators_era5l_optimal_cumulated = \
validate_pv_output_all(reference_non_tracking_cumulated,
                       era5l_optimal_cumulated,
                       Path('intermediate_results/indicators_era5l_optimal_cumulated.csv'))
indicators_era5l_tracking_cumulated =\
validate_pv_output_all(reference_tracking_cumulated,
                       era5l_tracking_cumulated,
                       Path('intermediate_results/indicators_era5l_tracking_cumulated.csv'))
indicators_merra2_optimal_cumulated = \
validate_pv_output_all(reference_non_tracking_cumulated,
                       merra2_optimal_cumulated,
                       Path('intermediate_results/indicators_merra2_optimal_cumulated.csv'))
indicators_merra2_tracking_cumulated = \
validate_pv_output_all(reference_tracking_cumulated,
                       merra2_tracking_cumulated,
                       Path('intermediate_results/indicators_merra2_tracking_cumulated.csv'))

In [15]:
indicators_era5l_optimal_des_cumulated = \
validate_pv_output_all(reference_non_tracking_des_cumulated,
                       era5l_optimal_des_rad_cumulated,
                       Path('intermediate_results/indicators_era5l_optimal_deseason_cumulated.csv'))
indicators_era5l_tracking_des_cumulated =\
validate_pv_output_all(reference_tracking_des_cumulated,
                       era5l_tracking_des_rad_cumulated,
                       Path('intermediate_results/indicators_era5l_tracking_deseason_cumulated.csv'))
indicators_merra2_optimal_des_cumulated = \
validate_pv_output_all(reference_non_tracking_des_cumulated,
                       merra2_optimal_des_rad_cumulated,
                       Path('intermediate_results/indicators_merra2_optimal_deseason_cumulated.csv'))
indicators_merra2_tracking_des_cumulated = \
validate_pv_output_all(reference_tracking_des_cumulated,
                       merra2_tracking_des_rad_cumulated,
                       Path('intermediate_results/indicators_merra2_tracking_deseason_cumulated.csv'))